In [1]:
import gurobipy as gp
from gurobipy import *

In [2]:
instance = './quercus_instances/answer0.txt'

In [6]:
with open(instance) as f:
    next(f)
    n, m = (int(item) for item in next(f).split())
    
    tasks = [[(int(line.split()[i]),int(line.split()[i+1])) 
              for i in range(0,m*2,2)] for line in f]
    print(tasks)

[[(0, 4), (1, 1), (2, 5)], [(1, 2), (2, 3), (1, 7)]]


In [4]:
Jobs = [job for job in range(n)]
J = [(job,task) for job in range(n) for task in range(m)]
K = [k for k in range(m)]

print("n jobs:", n)
print("m machines:", m)
print("\nSet of all jobs (Jobs):", Jobs) 
print("Set of all machines (K):", K)
print("\nSet of all operations (J):", J)

n jobs: 2
m machines: 3

Set of all jobs (Jobs): [0, 1]
Set of all machines (K): [0, 1, 2]

Set of all operations (J): [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]


In [7]:
#(P_j) dictionary to access processing time for each operation j [key]
p = {j:tasks[j[0]][j[1]][1] for j in J}
print("Processing times (P_j):", p)

# some large constant
M = sum(p.values())  #we set to sum of all p_j in instance
print("Large constant (M):", M)

Processing times (P_j): {(0, 0): 4, (0, 1): 1, (0, 2): 5, (1, 0): 2, (1, 1): 3, (1, 2): 7}
Large constant (M): 22


In [11]:
#set of operations on machine k out of all machines K
J_k = [[] for _ in range(m)]
for j in J:
        for k in K:
            if tasks[j[0]][j[1]][0]==k:
                J_k[k].append((j))
print(J_k)

[[(0, 0)], [(0, 1), (1, 0), (1, 2)], [(0, 2), (1, 1)]]


In [ ]:
#all operation pairs (j,i) s.t. i is constrained to be after j
#all precedence constraints
Epsilon = [(J[i-1],j) for i,j in enumerate(J) if i > 0 if j[0] == J[i-1][0]]
Epsilon

In [ ]:
#A will be set of all possible operation combinations in each J_k
A = [(j,i) for k in K for j in J_k[k] for i in J_k[k] if j !=i]

In [ ]:
model = gp.Model()          #create Gurobi model
model.params.TimeLimit = 60 #set time limit in seconds

In [ ]:
#Decision Variables
S = model.addVars(J, vtype=GRB.INTEGER, name="S_j")  #start time of operation j
z = model.addVars(A, vtype=GRB.BINARY, name="Z_ji")  #binary of if j before i 
Cmax = model.addVar(vtype=GRB.INTEGER, name="C_max") #makespan

In [ ]:
#1. Makespan Constraints
#Cmax must be largest end-time of any op j in set J (all operations)
model.addConstrs(Cmax >= (S[j] + p[j]) for j in J);

In [ ]:
#2. Precedence Constraints
#Start time of operation i must be greater than
#End time of operation j for all (j,i) pairs in Epsilon
#Epsilon is the set of all adjacent operation pairs in each job
model.addConstrs(S[i]>=(S[j]+p[j]) for j,i in Epsilon);

In [ ]:
#3. & #4. Resource Constraints
#for each operation pair (j,i) for each resource k, if j is before i
#then Start time of i is greater than finish time of j
#otherwise: Start time of j is greater than finish time of i
model.addConstrs(S[j] >= (S[i] + p[i] - M * z[j,i]) for j, i in A);
model.addConstrs(S[i] >= (S[j] + p[j] - M * (1-z[j,i])) for j,i in A);

In [ ]:
#5. Start time must be greater than 0
model.addConstrs(S[j] >= 0 for j in J);

In [ ]:
#6. Objective Function: MINIMIZE Cmax (makespan)
model.setObjective(Cmax, GRB.MINIMIZE);
model.optimize()

In [ ]:
makespan = model.Objval
schedule = {job:[int(S[j].x) for j in J if j[0]==job] for job in Jobs}
solution = (makespan, schedule)
print(solution)